In [4]:
import numpy as np 
import matplotlib.pyplot as plt
fold = '/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction/allData/prognosis/pre_treatment_hamd_reduction_50'

hb = np.load(fold + '/data.npy')
label = np.load(fold + '/label.npy')

print(label)

[0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0
 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1]


In [16]:
dmfc_dt = np.load('/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction/results/ML_results/DMFC/pre_post_treatment_hamd_reduction_50/Decision Tree_substract_result.npy', allow_pickle=True)

{'accuracies': {'HbT': 0.6304347826086957}, 'model_accuraies': {'HbO': [1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0], 'HbR': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'HbT': [1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]}, 'current_time_seed': 1706258708, 'num_time': 0, 'HB_TYPE_y_pred_and_y_test': {'HbO': [[array([0]), array([0])], [array([1]), array([0])], [array([0]), array([0])], [array([0]), array([0])], [array([0]), array([1])], [a

In [2]:
# pre_post_treatment_hamd_reduction_50

data = np.load('/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction/allData/prognosis/pre_post_treatment_hamd_reduction_50/data.npy')

print(data.shape)

(46, 52, 2502, 2)


In [28]:
tmp = '/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction/results/ML_results/DMFC/pre_post_treatment_hamd_reduction_50/Decision Tree_substract_result.npy'
tmp = np.load(tmp, allow_pickle=True).item()
print(tmp['0']['model_accuraies']['HbT'])

[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]


In [11]:
base_dmfc = np.load('/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction/allData/prognosis/DMFC/pre_treatment_hamd_reduction_50/data.npy')

base_T8_dmfc = np.load('/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction/allData/prognosis/DMFC/pre_post_treatment_hamd_reduction_50/data.npy')

print(base_dmfc.shape)
print(base_T8_dmfc.shape)


(65, 52, 52, 3)
(46, 52, 52, 3, 2)


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

hb_2d = np.reshape(hb, (hb.shape[0], -1))
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(hb_2d, label, test_size=0.5, random_state=42)

# Create a random forest classifier
rf_classifier = RandomForestClassifier()

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = rf_classifier.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.7272727272727273


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score
import numpy as np
# Initialize LeaveOneOut
loo = LeaveOneOut()

# Initialize the classifier
rf_classifier = RandomForestClassifier()

# List to store the accuracy for each fold
accuracies = []

# Loop over each train/test split
for train_index, test_index in loo.split(hb_2d):
    # Split the data into training and testing sets
    X_train, X_test = hb_2d[train_index], hb_2d[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # Train the classifier
    rf_classifier.fit(X_train, y_train)
    
    # Predict the label for the test set
    y_pred = rf_classifier.predict(X_test)
    
    # Calculate the accuracy for the current fold
    accuracy = accuracy_score(y_test, y_pred)
    
    # Append the accuracy to the list
    accuracies.append(accuracy)

# Calculate the average accuracy across all folds
average_accuracy = np.mean(accuracies)
print("LOOCV Accuracy:", average_accuracy)

LOOCV Accuracy: 0.7538461538461538


# 找到是哪些数据没有识别对 （注意看这里！！！）

In [13]:
for i, acc in enumerate(accuracies):
    if acc == 1:
        print(label[i])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# 你的数据预处理部分
hb_2d = np.reshape(hb, (hb.shape[0], -1))  # 假设hb是你的特征矩阵

# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(hb_2d, label, test_size=0.2, random_state=42)

# 初始化模型
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier()
}

# 存储每个模型的准确率
accuracies = {}

for name, model in models.items():
    print(f'current training {name}')
    model_accuraies = []
    # Loop over each train/test split
    for train_index, test_index in loo.split(hb_2d):
        # Split the data into training and testing sets
        X_train, X_test = hb_2d[train_index], hb_2d[test_index]
        y_train, y_test = label[train_index], label[test_index]
        
        # Train the classifier
        model.fit(X_train, y_train)
        
        # Predict the label for the test set
        y_pred = model.predict(X_test)
        
        # Calculate the accuracy for the current fold
        accuracy = accuracy_score(y_test, y_pred)
        
        # Append the accuracy to the list
        model_accuraies.append(accuracy)

    accuracies[name] = np.mean(model_accuraies)

# 比较模型性能
best_model = max(accuracies, key=accuracies.get)
print(f"Best performing model: {best_model} with accuracy: {accuracies[best_model]:.4f}")


current training Logistic Regression
current training Random Forest
current training SVM
current training KNN
current training Decision Tree
Best performing model: Decision Tree with accuracy: 0.8308


In [38]:
print(hb.shape)
hbo = hb[...,:hb.shape[2]//2]
hbr = hb[...,hb.shape[2]//2:]

(65, 52, 2502)


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# 你的数据预处理部分
hbo_2d = np.reshape(hbo, (hbo.shape[0], -1))  # 假设hb是你的特征矩阵

# 初始化模型
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier()
}

# 存储每个模型的准确率
accuracies = {}
model_accuraies = {}

for name, model in models.items():
    print(f'current training {name}')
    model_accuraies[name] = []
    
    # Loop over each train/test split
    for train_index, test_index in loo.split(hbo_2d):
        # Split the data into training and testing sets
        X_train, X_test = hbo_2d[train_index], hbo_2d[test_index]
        y_train, y_test = label[train_index], label[test_index]
        
        # Train the classifier
        model.fit(X_train, y_train)
        
        # Predict the label for the test set
        y_pred = model.predict(X_test)
        
        # Calculate the accuracy for the current fold
        accuracy = accuracy_score(y_test, y_pred)
        
        # Append the accuracy to the list
        model_accuraies[name].append(accuracy)

    accuracies[name] = np.mean(model_accuraies[name])

# # 比较模型性能
best_model = max(accuracies, key=accuracies.get)
print(f"Best performing model: {best_model} with accuracy: {accuracies[best_model]:.4f}")


current training Logistic Regression
current training Random Forest
current training SVM
current training KNN
current training Decision Tree
Best performing model: KNN with accuracy: 0.7846


In [45]:
for name, acc in accuracies.items():
    print(f'{name}: {acc:.4f}')

for name, acc in model_accuraies.items():
    print(f'{name}: {acc}')

Logistic Regression: 0.6769
Random Forest: 0.7692
SVM: 0.7692
KNN: 0.7846
Decision Tree: 0.7231
Logistic Regression: [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0]
Random Forest: [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0]
SVM: [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 

In [77]:
from scipy import stats
loocv_results = np.array(model_accuraies['KNN'])
print(loocv_results)
# 计算均值（平均准确率）和标准误差
mean_accuracy = np.mean(loocv_results)
se = stats.sem(loocv_results) # 标准误差

# 计算95%置信区间
confidence_interval = stats.norm.interval(0.95, loc=mean_accuracy, scale=se)

print(f"LOOCV Mean accuracy: {mean_accuracy:.4f}")
print(f"95% CI: {confidence_interval}")

[1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1.
 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0.]
LOOCV Mean accuracy: 0.7846
95% CI: (0.683900455067013, 0.8853303141637562)


In [91]:
def pre_post_read_hb_label(HB_TYPE, fold='/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction/allData/prognosis/pre_post_treatment_hamd_reduction_50'):
    # read data 
    hb = np.load(fold + '/data.npy')
    label = np.load(fold + '/label.npy')
    print(hb.shape)
    if HB_TYPE == 'HbO':
        hb = np.concatenate((hb[...,:hb.shape[2]//2, 0], hb[...,:hb.shape[2]//2, 1]), axis=2)
    elif HB_TYPE == 'HbR':
        hb = np.concatenate((hb[...,hb.shape[2]//2:, 0], hb[...,hb.shape[2]//2:, 1]), axis=2)
    elif HB_TYPE == 'HbO+HbR':
        hb = np.concatenate((hb[..., 0], hb[..., 1]), axis=-1)
    print(hb.shape)
    hb_2d = np.reshape(hb, (hb.shape[0], -1))
    return hb_2d, label

x, y = pre_post_read_hb_label('HbO')
print(x.shape)

(46, 52, 2502, 2)
(46, 52, 2502)
(46, 130104)


In [75]:
tmp = np.load('/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction/results/ML_results/ML_results.npy',allow_pickle=True)

print(tmp)
for x,y in tmp.item().items():
    print(x)
    for k,v in y.items():
        print(k,v)
    print('-----------------')

{'HbO_0': {'accuracies': {'KNN': 0.7846153846153846}, 'model_accuraies': {'KNN': [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0]}, 'current_time_seed': 1706164183, 'hb_type': 'HbO', 'num_time': 0, 'model_y_pred_and_y_test': {'KNN': [[array([0]), array([0])], [array([0]), array([0])], [array([0]), array([0])], [array([0]), array([0])], [array([0]), array([0])], [array([0]), array([1])], [array([0]), array([0])], [array([0]), array([0])], [array([0]), array([0])], [array([0]), array([1])], [array([0]), array([0])], [array([0]), array([0])], [array([0]), array([0])], [array([0]), array([0])], [array([0]), array([1])], [array([0]), array([0])], [array([0]), array([0])], [array([0]), array([0])], [array([1]), array([1

In [74]:
print(y_pred)

[1]
